Wykorzystamy zbiór danych pingwinów zebrany przez stację Palmer na Antarktydzie. Zestaw składa się z danych dotyczących 344 pingwinów znalezionych na trzech wyspach Archipelagu Palmera na końcu Półwyspu Antarktycznego. Na każdym pingwinie wykonuje się 4 pomiary numeryczne, podaje się jego gatunek, płeć i lokalizację. Zbiór danych zawiera 3 gatunki pingwinów i 11 wierszy brakujących danych.

In [89]:
from operator import concat

import pandas as pd
from sklearn.preprocessing import LabelEncoder

df = pd.read_csv("data/penguins_lter.csv", na_values=["NaN", ""])




1. Wygeneruj etykiety numeryczne dla kolumn "species", "island" i "sex".

In [90]:
encoders = {}
cols = ["Species", "Island", "Sex"]
df = df[cols].copy()
df = df.dropna().reset_index(drop=True)
for col in ["Species", "Island", "Sex"]:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    encoders[col] = le


df.head()

,Species,Island,Sex
0,0,2,2
1,0,2,1
2,0,2,1
3,0,2,1
4,0,2,2


2. Zbuduj (kNN) klasyfikator płci pingwina. - Podziel zbiór danych na zbiór uczący i zestaw testowy i porównaj klasyfikator  ze zbiorem testowym. (accuracy, F1-score) Liczba sąsiadów w KNeighborsClassifier sklearn jest określana za pomocą parametrów n_neighbors. Użyj n_neighbors = 3.

In [91]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, f1_score
df_new = df.copy()
Y = df.pop("Sex")
X = df

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train)

y_pred = knn.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

result = pd.DataFrame([["KNeighborsClassifier",accuracy, f1]], columns=["Model","Accuracy", "F1"])

result


,Model,Accuracy,F1
0,KNeighborsClassifier,0.41791,0.170213


3. Znajdź dobrą wartość dla n_neighbors w klasyfikatorze płci pingwina. - Uruchom szkolenie klasyfikatora w pętli i wydrukuj wynik dokładności dla różnych wartości n_neighbors =. Wartości do n_neighbors = 20 powinny wystarczyć.

In [92]:
rows = []

for n in range(1, 21):
    knn = KNeighborsClassifier(n_neighbors=n)
    knn.fit(X_train, y_train)
    y_pred = knn.predict(X_test)
    rows.append({"n_neighbors": n,
                 "accuracy": accuracy_score(y_test, y_pred),
                 "f1": f1_score(y_test, y_pred)})

tuning_sex = pd.DataFrame(rows)
tuning_sex

,n_neighbors,accuracy,f1
0,1,0.507463,0.571429
1,2,0.507463,0.571429
2,3,0.417910,0.170213
3,4,0.417910,0.170213
4,5,0.447761,0.000000
5,6,0.417910,0.170213
6,7,0.417910,0.170213
7,8,0.432836,0.457143
8,9,0.432836,0.457143
9,10,0.462687,0.571429


4. Zbuduj klasyfikator gatunków pingwinów i znajdź dobrą wartość dla n_neighbors w tym klasyfikatorze.

In [98]:
X_species = df.drop(columns=["Species"])
Y_species = df["Species"]

X_train, X_test, y_train, y_test = train_test_split(
    X_species, Y_species, test_size=0.2, random_state=42, stratify=Y_species
)

rows = []
for n in range(1, 21):
    knn = KNeighborsClassifier(n_neighbors=n)
    knn.fit(X_train, y_train)
    y_pred = knn.predict(X_test)
    rows.append({
        "n_neighbors": n,
        "accuracy": round(accuracy_score(y_test, y_pred),3),
        "f1_macro": round(f1_score(y_test, y_pred, average="macro"),3)
    })
tuning_species = pd.DataFrame(rows)
tuning_species

,n_neighbors,accuracy,f1_macro
0,1,0.687,0.661
1,2,0.493,0.422
2,3,0.687,0.661
3,4,0.433,0.201
4,5,0.687,0.661
5,6,0.433,0.201
6,7,0.627,0.468
7,8,0.627,0.468
8,9,0.627,0.468
9,10,0.627,0.468
